In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider
from solar_tracking.main import (
    calculate_declination,
    calculate_elevation,
    calculate_azimuth,
)
from solar_tracking.solar_panel import SolarPanel

In [2]:
def get_sun_path(latitude, day_of_year):
    declination = calculate_declination(day_of_year)

    hours = np.linspace(-180, 180, 361)  # Hour angles from -180 to 180
    elevations = []
    azimuths = []

    for h in hours:
        elev = calculate_elevation(latitude, declination, h)
        az = calculate_azimuth(latitude, declination, h, elev)

        if elev >= 0:
            elevations.append(elev)
            azimuths.append(az)

    return azimuths, elevations

In [3]:
def plot_panel_simulation(
    latitude=40.7128, day_of_year=172, hour_angle=0, panel_azimuth=0, panel_tilt=30
):
    # 1. Calculate Sun Position
    declination = calculate_declination(day_of_year)
    sun_elev = calculate_elevation(latitude, declination, hour_angle)
    sun_az = calculate_azimuth(latitude, declination, hour_angle, sun_elev)

    # 2. Calculate Incidence Angle
    panel = SolarPanel(
        latitude, 0, panel_azimuth, panel_tilt
    )  # Longitude doesn't affect incidence calculation here
    incidence = panel.calculate_incidence_angle(sun_az, sun_elev)

    # 3. Get Sun Path for plotting
    path_az, path_elev = get_sun_path(latitude, day_of_year)

    # 4. Plotting
    plt.figure(figsize=(12, 7))

    # Plot Sun Path
    plt.plot(path_az, path_elev, label="Sun Path", color="orange", alpha=0.7)

    # Plot Sun Position
    if sun_elev >= 0:
        plt.scatter([sun_az], [sun_elev], color="red", s=150, label="Sun", zorder=10)
        plt.text(
            sun_az,
            sun_elev + 3,
            f"Sun\nAz: {sun_az:.1f}°\nEl: {sun_elev:.1f}°",
            ha="center",
            fontsize=9,
        )
    else:
        plt.text(0, 45, "Sun is below horizon", ha="center", fontsize=14, color="gray")

    # Plot Panel Normal Vector (approximate visualization)
    # The normal vector points towards (panel_azimuth, 90 - panel_tilt)
    normal_elev = 90 - panel_tilt
    plt.scatter(
        [panel_azimuth],
        [normal_elev],
        color="blue",
        marker="^",
        s=150,
        label="Panel Normal",
        zorder=9,
    )
    plt.text(
        panel_azimuth,
        normal_elev - 5,
        f"Panel Normal\nAz: {panel_azimuth}°\nTilt: {panel_tilt}°",
        ha="center",
        color="blue",
        fontsize=9,
    )

    # Display Info
    info_text = (
        f"Incidence Angle: {incidence:.2f}°\n"
        f"Efficiency: {max(0, math.cos(math.radians(incidence))):.1%}"
    )
    plt.text(
        0.02,
        0.95,
        info_text,
        transform=plt.gca().transAxes,
        fontsize=12,
        bbox=dict(facecolor="white", alpha=0.8),
    )

    plt.title(
        f"Solar Panel Simulation (Lat: {latitude}°, Day: {day_of_year})\nHour Angle: {hour_angle}°"
    )
    plt.xlabel("Azimuth (degrees)")
    plt.ylabel("Elevation (degrees)")
    plt.ylim(0, 90)
    plt.xlim(-180, 180)
    plt.grid(True, alpha=0.3)
    plt.legend(loc="upper right")
    plt.show()

In [4]:
interact(
    plot_panel_simulation,
    latitude=FloatSlider(
        min=-90, max=90, step=0.1, value=40.7128, description="Latitude"
    ),
    day_of_year=IntSlider(min=1, max=365, step=1, value=172, description="Day of Year"),
    hour_angle=FloatSlider(
        min=-180, max=180, step=1, value=0, description="Hour Angle"
    ),
    panel_azimuth=FloatSlider(
        min=-180, max=180, step=1, value=0, description="Panel Azimuth"
    ),
    panel_tilt=FloatSlider(min=0, max=90, step=1, value=30, description="Panel Tilt"),
);

interactive(children=(FloatSlider(value=40.7128, description='Latitude', max=90.0, min=-90.0), IntSlider(value…